In [5]:
from tagger.models.received_item import ReceivedItem, ReceivedItemStatus
from tagger.core.mongo.models.order import OrderCodeMap, Order
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import pandas as pd
from bson import ObjectId

def get_row(r: ReceivedItem):
    payment = r.order.get_payment()
    return {
        "item_name": r.item_name,
        "size": r.size,
        "order_url": f"https://office.alloff.co/orders/{r.order_id}",
        "current_order_status": r.order.orderstatus,
        "brand_name": r.product.brand.korname,
        "order_code": r.order.code,
        "order_created": r.order.created,
        "buyername": payment.buyername
    }

ImportError: cannot import name 'ObjectId' from 'mongoengine' (/Users/stephenj/Library/Caches/pypoetry/virtualenvs/alloff-backoffice-server-V9pDS2iq-py3.9/lib/python3.9/site-packages/mongoengine/__init__.py)

In [25]:
rows = [get_row(r) for r in ReceivedItem.objects.all()]
    

In [26]:
df = pd.DataFrame(rows)

In [22]:
with open("../Desktop/test.xlsx", "wb") as f:
    df.to_excel(f)

In [27]:
 df

,item_name,size,order_url,current_order_status,brand_name,order_code,order_created,buyername
0,꽈배기 짜임 포인트 양말,FREE,https://office.alloff.co/orders/615d7eac22b2fd...,CANCEL_FINISHED,나이스클랍,,2021-10-06 10:47:08.801,테스트이석민
1,Tiam,0,https://office.alloff.co/orders/616a04ef17fc2e...,DELIVERY_PREPARING,몽클레르,,2021-10-15 22:47:11.189,류일향
2,Frankby quilted jacket(블랙),XS,https://office.alloff.co/orders/616ac84717fc2e...,CANCEL_REQUESTED,버버리,,2021-10-16 12:40:39.110,조현주
3,Wool and cashmere cardigan(Camel beige),M,https://office.alloff.co/orders/616bfc4c17fc2e...,DELIVERY_PREPARING,막스마라,,2021-10-17 10:34:52.678,이남이
4,캐시미어 니트 롱 드레스,90(F),https://office.alloff.co/orders/616d707617fc2e...,CANCEL_PENDING,시스템,,2021-10-18 13:02:46.986,박미옥
...,...,...,...,...,...,...,...,...
135,[로렌 랄프로렌] 브로드클로스 블루종 슬리브 셔츠,M(66),https://office.alloff.co/orders/6182768926685f...,PRODUCT_PREPARING,폴로 랄프로렌,ORD-EFU79,2021-11-03 11:46:17.742,김유미
136,스티치 니트 스웻셔츠,85,https://office.alloff.co/orders/6182834026685f...,PAYMENT_FINISHED,에스제이에스제이,ORD-94DYH,2021-11-03 12:40:32.342,김은진
137,Ashurst quilted jacket(블랙),S,https://office.alloff.co/orders/6183173b26685f...,PAYMENT_FINISHED,버버리,ORD-3NVHL,2021-11-03 23:11:55.234,김태연
138,벨티드 플랩 코트(카멜),82(55),https://office.alloff.co/orders/6183372e26685f...,PAYMENT_FINISHED,씨씨콜렉트,ORD-BX36R,2021-11-04 01:28:14.399,박선정


In [36]:
items = Order.objects(
    id=OrderCodeMap.objects(code="ORD-CGA6M").first().orderid
).first().orders

82(55) 1 워싱 코튼 패딩 점퍼(그레이)  None


In [25]:
from bson import ObjectId
inventories = [
    "617b49a017fc2e5a1f9ead04",
    "61701beb17fc2e5a1f9e9597",
    "61796a8017fc2e5a1f9eaa39",
    "61776a9017fc2e5a1f9ea5a4",
    "617cb70617fc2e5a1f9eaf78",
    "617e860d17fc2e5a1f9eb12e",
    "617570b817fc2e5a1f9ea18f",
    "6172505a17fc2e5a1f9e9a55",
    "616e1b5917fc2e5a1f9e91e6",
    "616e094917fc2e5a1f9e91bd",
    "616d707617fc2e5a1f9e907d",
    "6178fa2017fc2e5a1f9ea859",
    "6173f22f17fc2e5a1f9e9eaa",
    "6171131b17fc2e5a1f9e9754",
    "61715b9817fc2e5a1f9e9842",
    "6175fb2e17fc2e5a1f9ea2a7",
    "618145d826685ff051754d89",
    "617cb59e17fc2e5a1f9eaf75",
    "6180fd6d26685ff051754c51",
    "6178d5d217fc2e5a1f9ea810",
    "6171f2c617fc2e5a1f9e997b",
    "6177e05817fc2e5a1f9ea668",
]

inv_ords = [x for x in Order.objects(id__in=[ObjectId(i) for i in inventories])]

products = {}
alloffproducts = {}
for i in inv_ords:
    for item in i.orders:
        if item.alloffproduct is not None:
            key = (f"{item.alloffproduct.name.strip()}___{item.alloffproduct.brand.keyname}", item.size)
            if key in alloffproducts:
                alloffproducts[key].append(str(i.id))
            else:
                alloffproducts[key] = [str(i.id)]
        if item.product is not None:
            key = (f"{item.product.name.strip()}___{item.alloffproduct.brand.keyname}", item.size)
            if key in products:
                products[key].append(str(i.id))
            else:
                products[key] = [str(i.id)]

In [26]:
hungry_orders = Order.objects(orderstatus__in=[
    "PRODUCT_PREPARING",
    "PAYMENT_FINISHED",
    "DELIVERY_PREPARING"
], ordertype="TIMEDEAL_ORDER").all()

shippables = []


for h in hungry_orders:
    key = (f"{h.orders[0].alloffproduct.name.strip()}___{h.orders[0].alloffproduct.brand.keyname}", h.orders[0].size)
    print(key)
    if key in alloffproducts:
        shippables.append(h)

('Tiam___MONCLER', '0')
('Wool and cashmere cardigan(Camel beige)___MAXMARA', 'M')
('[택 제거 상품] Amore(Nero)___MAXMARA', '40')
('네이비 체크 셔츠___VINCE', 'XS')
('네이비 체크 셔츠___VINCE', 'M')
('캐시미어 롱 가디건(블랙)___THEORY', 'S')
('캐시미어 롱 가디건(차콜)___THEORY', 'P')
('[택 제거 상품] BETTA(Brown)___MAXMARA', '38')
('[택 제거 상품] JULIA(Black)___MAXMARA', '38')
('더블 M 머플러___MAJE', 'one size')
('울 캐시미어 블렌드 니트___VINCE', 'L')
('캐시미어 더블 코트___SYSTEM', '82(55) ')
('오버사이즈 와이드 칼라 코트(라임)___CCCOLLECT', '82(55)')
('[택 제거 상품] Teca___MAXMARA', '42')
('[Weekend MaxMara] PATNER(Camel)___MAXMARA', '42')
('[MEN] (7 colors) 울 캐시미어 스웨터___POLO', '차콜/M')
('컬러 블록 페이크 무스탕 자켓(아이보리)___CCCOLLECT', '82(55)')
('[로렌 랄프로렌] 서플리스 데님 미디 드레스___POLO', '6(66)')
('네이비 단추 FIRE 니트___SANDRO', '3')
('네이비 CLUB 맨투맨___SANDRO', '3')
('[MEN] (7 colors) 울 캐시미어 스웨터___POLO', '네이비/L')
('[택 제거 상품] Teca___MAXMARA', '42')
('Harrowden(올리브)___BURBERRY', 'L')
('Ashby canvas bucket bag(블랙/미듐)___BURBERRY', 'M')
('Harrowden(밍크 그레이)___BURBERRY', 'S')
('Torto sweatshirt(그레이)__

In [27]:
for s in shippables:
    p = s.get_payment()
    print(str(s.id), p.name, p.buyername, s.orders[0].size, s.created, s.code)

61812c7d26685ff051754cee 핸드메이드 더블 코트  김명수 82(55) 2021-11-02 12:18:05.424000 ORD-LXPJU


In [28]:
alloffproducts

{('캐시미어 니트 롱 드레스___SYSTEM', '90(F)'): ['616d707617fc2e5a1f9e907d',
  '616e094917fc2e5a1f9e91bd',
  '616e1b5917fc2e5a1f9e91e6',
  '6172505a17fc2e5a1f9e9a55',
  '617570b817fc2e5a1f9ea18f'],
 ('[빈폴스포츠] 듀얼 후드 코어 쇼트 다운(옐로우)___BEANPOLEL',
  '85'): ['61701beb17fc2e5a1f9e9597'],
 ('핸드메이드 더블 코트___TIME', '82(55)'): ['6171131b17fc2e5a1f9e9754',
  '61715b9817fc2e5a1f9e9842'],
 ('[기획] 핸드메이드 벨티드 코트___SJSJ', '76(44)'): ['6171f2c617fc2e5a1f9e997b'],
 ('핸드메이드 더블 코트___SYSTEM', '82(55)'): ['6173f22f17fc2e5a1f9e9eaa',
  '6175fb2e17fc2e5a1f9ea2a7'],
 ('아노락 퀼팅 점퍼(오렌지)___THECASHMERE', '82(F)'): ['61776a9017fc2e5a1f9ea5a4'],
 ('볼륨 소매 니트 풀오버(민트)___DEWL', 'FREE(99)'): ['6177e05817fc2e5a1f9ea668'],
 ('워싱 코튼 패딩 점퍼(그레이)___CCCOLLECT', '82(55)'): ['6178d5d217fc2e5a1f9ea810'],
 ('컬러 블록 페이크 무스탕 자켓(그레이)___CCCOLLECT', '82(55)'): ['6178fa2017fc2e5a1f9ea859'],
 ('스티치 디테일 핸드메이드 코트(그린)___CCCOLLECT', '88(66)'): ['61796a8017fc2e5a1f9eaa39'],
 ('[Re-order] 알파카 블렌드 니트 가디건(스모크 핑크)___TIME',
  '90(F)'): ['617b49a017fc2e5a1f9ead04'

In [80]:
o = Order.objects(id=OrderCodeMap.objects(code="ORD-E7RP4").first().orderid).first()
p = o.get_payment()
print(p.name, p.buyername, str(o.id))

하이넥 울 니트 탑  박미화  61813fbd26685ff051754d3c


In [81]:
can_ship = [
    "617ff99b17fc2e5a1f9eb3a8",
    "617ff99b17fc2e5a1f9eb3a8",
    "617114a117fc2e5a1f9e9768",
    "617e6bf517fc2e5a1f9eb114",
    "6180ffde26685ff051754c66",
    "617e4d7717fc2e5a1f9eb0f4",
    "618139cf26685ff051754d2a",
    "6180fe1b26685ff051754c57",
    "6180fe8626685ff051754c5a",
    "6180fd6d26685ff051754c51",
    "6181bbf926685ff051754e40",
    "6181415426685ff051754d4d",
    "61813fbd26685ff051754d3c",
    "618145d826685ff051754d89",
    "617e965017fc2e5a1f9eb18e",
    "61792ac017fc2e5a1f9ea8f0",
    "6178c83d17fc2e5a1f9ea7c6",
    "61812a1526685ff051754cdd",
    "618116fc26685ff051754ca1",
    "617a086e17fc2e5a1f9eaaba",
    "617cad7017fc2e5a1f9eaf6b",
    "6178d5d217fc2e5a1f9ea810",
]

can_ship_orders = Order.objects(id__in=[ObjectId(x) for x in can_ship]).all()

In [82]:
for c in can_ship_orders:
    print(c.orderstatus, c.id)

DELIVERY_PREPARING 617114a117fc2e5a1f9e9768
CANCEL_FINISHED 6178c83d17fc2e5a1f9ea7c6
CANCEL_FINISHED 6178d5d217fc2e5a1f9ea810
DELIVERY_PREPARING 61792ac017fc2e5a1f9ea8f0
DELIVERY_PREPARING 617a086e17fc2e5a1f9eaaba
DELIVERY_PREPARING 617cad7017fc2e5a1f9eaf6b
DELIVERY_PREPARING 617e4d7717fc2e5a1f9eb0f4
DELIVERY_STARTED 617e6bf517fc2e5a1f9eb114
PRODUCT_PREPARING 617e965017fc2e5a1f9eb18e
DELIVERY_PREPARING 617ff99b17fc2e5a1f9eb3a8
CANCEL_FINISHED 6180fd6d26685ff051754c51
DELIVERY_PREPARING 6180fe1b26685ff051754c57
DELIVERY_PREPARING 6180fe8626685ff051754c5a
DELIVERY_PREPARING 6180ffde26685ff051754c66
DELIVERY_PREPARING 618116fc26685ff051754ca1
DELIVERY_PREPARING 61812a1526685ff051754cdd
DELIVERY_PREPARING 618139cf26685ff051754d2a
DELIVERY_PREPARING 61813fbd26685ff051754d3c
DELIVERY_PREPARING 6181415426685ff051754d4d
DELIVERY_PREPARING 618145d826685ff051754d89
DELIVERY_PREPARING 6181bbf926685ff051754e40
